In [36]:
import tweepy
import os.path as path
import numpy as np
import nltk.tokenize as tk
import math
import scipy.stats as stats
import pandas as pd
import huggingface_hub
from scipy.stats import pearsonr
from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM
import torch
from transformers import BertModel
import sklearn.cluster
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import random


In [37]:
def get_all_examples():
    """Load all examples from a CSV file using pandas and return those that contain text.

    Returns:
        A list of tweets (or data) that contain text.
    """
    save_file = './twitter_emoji.csv'

    if path.exists(save_file):
        df = pd.read_csv(save_file, lineterminator = '\n')
        ids_to_examples = df[df['Text'].notna()].to_dict(orient='records')  # Convert to list of dicts
    else:
        print('Could not find tweets, so returning an empty list!')
        ids_to_examples = []

    return ids_to_examples

In [38]:
def get_tweets(raw_examples):
  tweets = []
  for tweet in raw_examples:
    for (key, value) in tweet.items():
      tweets.append(value)
  return tweets

In [39]:
import re

# Input tweet
tweet = """🧡@KeplerHomes AirdropBox event for #Arbitrum ecological users is here. A total of 550,000 addresses are eligible for #airdrop, and 5 types of AirDropbox with different scarcity can be issued.

💙Invitation code: 52DC39
🏆Airdrop Portal:👉 https://t.co/fudohu97uV"""

# gets rid of punctuation and https
def remove_gack(words):
  punctation = ".,/';:[]\-=`~!@#$%^&*()_+{}|<>?"
  for punc in punctation:
    while punc in words:
      words.remove(punc)
  while "https" in words:
    words.remove("https")
  return words


# Convert tweet into an array of words, keeping emojis
def tweet_to_words_with_emojis(tweet):
    # Match words, hashtags, mentions, URLs, numbers, and emojis
    words = re.findall(r'\w+|#\w+|@\w+|https?://\S+|[^\w\s]', tweet)
    words = remove_gack(words)
    return words

# Process the tweet
words_array = tweet_to_words_with_emojis(tweet)

# Print the result
print(words_array)


['🧡', '@KeplerHomes', 'AirdropBox', 'event', 'for', '#Arbitrum', 'ecological', 'users', 'is', 'here', 'A', 'total', 'of', '550', '000', 'addresses', 'are', 'eligible', 'for', '#airdrop', 'and', '5', 'types', 'of', 'AirDropbox', 'with', 'different', 'scarcity', 'can', 'be', 'issued', '💙', 'Invitation', 'code', '52DC39', '🏆', 'Airdrop', 'Portal', '👉', 't', 'co', 'fudohu97uV']


In [40]:
raw_tweets = get_all_examples()
tweets = get_tweets(raw_tweets)
token_tweets = []
for tweet in tweets:
  token_tweets.append(tweet_to_words_with_emojis(tweet))

(token_tweets[:10])

[['🧡',
  '@KeplerHomes',
  'AirdropBox',
  'event',
  'for',
  '#Arbitrum',
  'ecological',
  'users',
  'is',
  'here',
  'A',
  'total',
  'of',
  '550',
  '000',
  'addresses',
  'are',
  'eligible',
  'for',
  '#airdrop',
  'and',
  '5',
  'types',
  'of',
  'AirDropbox',
  'with',
  'different',
  'scarcity',
  'can',
  'be',
  'issued',
  '💙',
  'Invitation',
  'code',
  '52DC39',
  '🏆',
  'Airdrop',
  'Portal',
  '👉',
  't',
  'co',
  'fudohu97uV'],
 ['Remember',
  'success',
  'in',
  'online',
  'business',
  'is',
  'a',
  'marathon',
  'not',
  'a',
  'sprint',
  'Keep',
  'at',
  'it',
  'stay',
  'focused',
  'and',
  'success',
  'will',
  'come',
  '"',
  '#patience',
  '#onlinebusiness',
  '#success',
  'For',
  'more',
  'tips',
  'and',
  'Strategies',
  'follow',
  'me',
  '👉',
  '@coach_lawrence1',
  't',
  'co',
  'IvtL9Om86J'],
 ['@occupied_9',
  '@Rhiannon_clare_',
  '@FightHaven',
  'Thanks',
  'for',
  'the',
  'update',
  'the',
  'sh',
  't',
  'country',
  '

In [42]:
from gensim.models import Word2Vec

model = Word2Vec(token_tweets, min_count=1)
print(model.wv['🧡'])

[-0.33157292 -0.06214472  0.0272709   0.17310256  0.2832343  -0.39879048
 -0.11605591  0.8421823  -0.36212835 -0.4865224  -0.27863005 -0.56560814
  0.01343476  0.29526314  0.15272208 -0.17462778 -0.06653097 -0.35927102
  0.225426   -0.9609891   0.21114331  0.3280024   0.6068373  -0.06477914
 -0.07230147  0.23319519 -0.3974869   0.1482561  -0.29580605  0.0898007
  0.46139905 -0.17867513  0.33925295 -0.2853096  -0.3852568   0.14814863
  0.25440472  0.03648861 -0.17999403 -0.44507504 -0.05476517 -0.30308816
 -0.4794207   0.09262225  0.3439464  -0.22689356 -0.30501813 -0.01283462
  0.19546518  0.5567547  -0.2031824   0.11730916  0.17309612 -0.03136833
 -0.12485295  0.3709493   0.07905839  0.01404483  0.15611233  0.1572443
 -0.2087084  -0.03560617  0.09005549 -0.29032126 -0.4527      0.39569703
  0.04858784  0.49383834 -0.40349814  0.60064554  0.05411586  0.2583684
  0.50795126  0.03121199  0.25758913  0.11574158  0.30970752  0.09708289
 -0.09737311 -0.04479466 -0.04749396 -0.08024704  0.05